In [1]:
from functions.test_algorithms import *
from functions.random_search import * 
from slim_gsgp_lib.datasets.data_loader import *
import pickle

datasets = [globals()[i] for i in globals() if 'load' in i][2:]

#### Settings

In [2]:
pop_size = 100 
n_iter = 100
n_iter_rs = 50
n_iter_test = 30
p_train = 0.7

#### SLIM 

In [5]:
datasets[13:]

[<function slim_gsgp_lib.datasets.data_loader.load_ld50(X_y=True)>,
 <function slim_gsgp_lib.datasets.data_loader.load_ppb(X_y=True)>,
 <function slim_gsgp_lib.datasets.data_loader.load_bioav(X_y=True)>]

In [6]:
for dataset_loader in tqdm(datasets[13:]):
    X, y = dataset_loader()
    dataset_name = dataset_loader.__name__.split('load_')[1]

    # Test if the best hyperparameters are valid
    try: 
        with open(f"best_params/best_slim_{dataset_name}_{pop_size}_{n_iter}_True_just_strmut.pkl", 'rb') as f:
            best_params = pickle.load(f)
            print(f"Best hyperparameters for {dataset_name} already exist!")
    except:
        best_params = {}
            
    # Perform random search for both scaled and unscaled versions
    if best_params == {}:
        print(f"Performing random search for {dataset_name}...")
        best_params = random_search_slim(X, y, dataset_name, scale=True, p_train=p_train,
                                            iterations=n_iter_rs, pop_size=pop_size, n_iter=n_iter,
                                            struct_mutation=True, show_progress=False, x_o=False, identifier='just_strmut')
        
        print(f"Random search for {dataset_name} completed!")
    
    # Initialize dictionaries for scaled and unscaled results
    metrics = ['rmse_', 'mape_', 'nrmse_', 'r2_', 'mae_', 'std_rmse_', 'time_stats', 'train_fit', 'test_fit', 'size']
    results = {metric: {} for metric in metrics}

    for algorithm in best_params:
        # Retrieve the best hyperparameters for testing
        args = best_params[algorithm]

        # Test SLIM 
        rm, ma, nrmse, r2, mae, std_rmse, time, train, test, size = test_slim(
            X=X, y=y, args_dict=args, dataset_name=dataset_loader.__name__,
            ms_lower=0, ms_upper=1, n_elites=1,
            iterations=n_iter_test, scale=True, algorithm=algorithm,
            verbose=0, p_train=p_train, show_progress=False,
        )
        
        # Initialize storage for each algorithm if not already present
        for metric in metrics:
            if algorithm not in results[metric]:
                results[metric][algorithm] = []

        # Store results
        results['rmse_'][algorithm].append(rm)
        results['mape_'][algorithm].append(ma)
        results['nrmse_'][algorithm].append(nrmse)
        results['r2_'][algorithm].append(r2)
        results['mae_'][algorithm].append(mae)
        results['std_rmse_'][algorithm].append(std_rmse)
        results['time_stats'][algorithm].append(time)
        results['train_fit'][algorithm].append(train)
        results['test_fit'][algorithm].append(test)
        results['size'][algorithm].append(size)        

        print(f"Results for {algorithm} on {dataset_name} saved!")

    # Save the results to disk
    with open(f"results/SLIM/{dataset_name}_just_strmut.pkl", 'wb') as f:
        pickle.dump(results, f)

    print(f"Results for {dataset_name} saved!")
    print("---------------------------------------------------")

  0%|          | 0/3 [00:00<?, ?it/s]

Performing random search for ld50...
Random search for ld50 completed!
Results for SLIM+SIG2 on ld50 saved!
Results for SLIM*SIG2 on ld50 saved!
Results for SLIM+ABS on ld50 saved!
Results for SLIM*ABS on ld50 saved!
Results for SLIM+SIG1 on ld50 saved!


 33%|███▎      | 1/3 [1:04:37<2:09:15, 3877.61s/it]

Results for SLIM*SIG1 on ld50 saved!
Results for ld50 saved!
---------------------------------------------------
Performing random search for ppb...
Random search for ppb completed!
Results for SLIM+SIG2 on ppb saved!
Results for SLIM*SIG2 on ppb saved!
Results for SLIM+ABS on ppb saved!
Results for SLIM*ABS on ppb saved!
Results for SLIM+SIG1 on ppb saved!


 67%|██████▋   | 2/3 [2:15:56<1:08:33, 4113.52s/it]

Results for SLIM*SIG1 on ppb saved!
Results for ppb saved!
---------------------------------------------------
Performing random search for bioav...
Random search for bioav completed!
Results for SLIM+SIG2 on bioav saved!
Results for SLIM*SIG2 on bioav saved!
Results for SLIM+ABS on bioav saved!
Results for SLIM*ABS on bioav saved!
Results for SLIM+SIG1 on bioav saved!


100%|██████████| 3/3 [3:16:21<00:00, 3927.06s/it]  

Results for SLIM*SIG1 on bioav saved!
Results for bioav saved!
---------------------------------------------------
